In [4]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from LPRnet.LPRnet_separable import *
from generator import DataGenerator
import time
PROJECT_NAME = "LPRnet_keras"
MODEL_NAME = "ccpd001"

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1

In [7]:
import glob
import cv2

real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')

data = []
labels = []
val_data = []
val_labels = []

for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))/256
    data.append(image)
    labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

for file in real_images_val:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    val_data.append(image)
    val_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

training_set = np.array(data,dtype=np.float32)
training_labels = np.array(labels)
ragged = tf.ragged.constant(training_labels).to_tensor()
real_dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).batch(64)


C:\Users\carlos\AppData\Local\Temp/ipykernel_15412/2036217571.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_labels = np.array(labels)


In [5]:
if os.path.exists("trained_models//{}".format(MODEL_NAME)):
    model = keras.models.load_model(
        "trained_models//{}".format(MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
    )
    print("loading model")
else:
    model = LPRnet()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss =CTCLoss)
    model.build((1,24,94,3))
    print("creating model")

loading model


In [9]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).batch(64)
    return dataset
test_dataset = generate_dataset()

C:\Users\carlos\AppData\Local\Temp/ipykernel_15412/2355684483.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_labels = np.array(gen_labels)


In [6]:

TFLITE_PATH = 'tflite_models'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()


with open("./{}/{}.tflite".format(TFLITE_PATH,MODEL_NAME), 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpzxjt0pnw\assets


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpzxjt0pnw\assets


In [14]:
interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\{}.tflite".format(MODEL_NAME))
#interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\keras_lprnet_separable.tflite")
import numpy as np
import time 
import glob
import cv2
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], (1, 24, 94, 3))
interpreter.allocate_tensors()
#real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\data\\marty\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\datasets\\custom\\CCPD-PLATES-TEST\\*.png')
start = time.time()
ctr = 0
batch_images=[]
batch_labels = []
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    batch_labels.append(label)
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = test_image.astype(np.float32)
    batch_images.append(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    decoded = keras.backend.ctc_decode(output_data,(24,),greedy=False)
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        print(text, " "+ label)
        ctr += 1
print(time.time()-start)
print(ctr,len(real_images))

A015E6  A015E6
A01AB7  A01AB7
A01W85  A01W85
A02A68  A02A68
A02W76  A02W76
A04H08  A04H08
A056S1  A056S1
A05V07  A05V07
A062A5  A062A5
A06MZ0  A06MZ0
A076R3  A076R3
A08M23  A08M23
A09K65  A09K65
A0U120  A0U120
A10X59  A10X59
A12P97  A12P97
A132Q0  A132Q0
A15058  A15058
A1F160  A1F160
A1P508  A1P508
A1R418  A1R418
A1X328  A1X328
A202H1  A202H1
A20X29  A20X29
A20X50  A20X50
A23J21  A23J21
A263A1  A263A1
A267P7  A267P7
A271W3  A271W3
A27Y20  A27Y20
A280M0  A280M0
A295U9  A295U9
A2S053  A2S053
A313J0  A313J0
A32M58  A32M58
A34026  A34026
A34J10  A34J10
A351E3  A351E3
A35M15  A35M15
A362Q8  A362Q8
A36S83  A36S83
A37P59  A37P59
A381B1  A381B1
A3T721  A3T721
A45R95  A45R95
A47094  A47094
A4D090  A4D090
A50S68  A50S68
A53C51  A53C51
A562F1  A562F1
A56414  A56414
A570L3  A570L3
A575Y0  A575Y0
A581Q7  A581Q7
A582M5  A582M5
A638Y0  A638Y0
A63T52  A63T52
A6586A  A6586A
A65N85  A65N85
A67262  A67262
A67A86  A67A86
A67X51  A67X51
A6J369  A6J369
A6L216  A6L216
A6W528  A6W528
A706X1  A706X1
A70P03  A7

In [9]:
tflite_interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\{}.tflite".format(MODEL_NAME))
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (193, 24, 94, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (193, 24,36))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()
print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])
start = time.time()
batch = np.array(batch_images)
tflite_interpreter.set_tensor(input_details[0]['index'], batch)
tflite_interpreter.invoke()
output_data = tflite_interpreter.get_tensor(output_details[0]['index'])
ctr = 0
for k,v in enumerate(output_data):
        text = ""
        print(v.shape)
        undecoded=np.expand_dims(v,axis=0)
        decoded = keras.backend.ctc_decode(undecoded,(24,),greedy=True)
        for j in np.array(decoded[0]).reshape(24):
            if j >-1:
                text += DECODE_DICT[j]
        if batch_labels[k] == text:
            print(text)
            ctr += 1
print(ctr)
print(time.time()-start)
print(ctr,len(real_images))

== Input details ==
name: serving_default_inputs:0
shape: [193  24  94   3]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [193  24  36]
type: <class 'numpy.float32'>
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
DAF9190
(24, 36)
DAJ1083
(24, 36)
(24, 36)
DCP7079
(24, 36)
NBU4828
(24, 36)
NCG1221
(24, 36)
NCH8029
(24, 36)
(24, 36)
(24, 36)
NFX7563
(24, 36)
NGS2592
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
CAQ6650
(24, 36)
CPY711
(24, 36)
DAG3746
(24, 36)
DAH3150
(24, 36)
(24, 36)
DAL7407
(24, 36)
(24, 36)
(24, 36)
DCP5431
(24, 36)
DCP7323
(24, 36)
NA36503
(24, 36)
NAE1322
(24, 36)
NAE2008
(24, 36)
(24, 36)
NBA4750
(24, 36)
NBK6857
(24, 36)
NBP3395
(24, 36)
NBR9407
(24, 36)
NC04583
(24, 36)
NCD4761
(24, 36)
NCM8042
(24, 36)
NCM9147
(24, 36)
NDE9713
(24, 36)
NDF2712
(24, 36)
NDJ3111
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
NED3275
(24, 36)
(24, 36)
NGJ4631
(24, 36)
(24, 36)
(24, 36)
(24, 36)
CA

In [19]:
MODEL_PATH = 'trained_models'
TFLITE_PATH = 'tflite_models'
model = keras.models.load_model(
    os.path.join(MODEL_PATH, MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
)

In [10]:
import glob
import cv2
real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\*\\*\\*.png')
real_new_plates =glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\TFODCourse\\test frames\\autolabelled\\*.png')
ctr = 0
batch = []
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace('O',str(0))
    image = cv2.imread(file).astype('float32')
    test_image = cv2.resize(image,(94,24))/256
    batch.append(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    preds = model.predict(test_image) 
    decoded = tf.keras.backend.ctc_decode(preds,(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text or label == text[:-1] or label == text[1:]:
        ctr += 1
    else:
        print(text," "+ label)
print(ctr)
print(len(real_images))

AXA302B  AXA3028
DAC581B  DAC5818
DD23624AA  DD23624
HQNXT20AA  F0W128
NE0141  NED1441
NG8Z516A  NGB2516
P1E5ZAAA  PIE527
ATF2TTMAA  TIS271
UDSL791A  USL794
WCP28TA  WCP291
KWC079TA  WCQ791
MUR28P3AA  WQK
YZ0G54  YZ9654
ZMM74BA  ZMM748
RPV720TAA  ZPV201
LB02158LA  DU2156
AACGB84  AAC6884
AAN88T4  AAN8814
ABZL4748A  ABL4748
ABT3G0B  ABT3609
AHA320B  AHA3208
AJA4T35  AJA4135
AJA4T40A  AJA4140
DAB4533AA  DAB4533
AH5784  DAH5784
0AN5585  DAN558B
LDAN98B50  DAN9650
DMM572AA  DMW572
EH5822A  EH5022
MV3783AA  MV3783
4WCJ4567  NCJ4567
NCCV535A  NDQ536
LNKT2586A  NK1258
6NK1258XA  NK1258
N0855QA  N08559
P89ZT07AA  P4Z107
PFJ878UA  PFJ870
RJNHBT3JVAA  RJN813
RN01GEHL  RND688
SJYXX799A  SJX799
TGGBPG  TGG996
TYNV552  TYW552
UAE305A  UAE505
UBK95TA  UBK951
UHR83TA  UHR831
UHYST5AA  UHY915
ULB5G2  ULB562
LPH549A  UPH549
VCEZ00A  VCE200
VCE2GQA  VCE200
RVDUMFZA  VDU792
MHJB22QA  WJB229
R4GML12ZJ9VA  XRC239
XTTB8BAA  XTT888
Z4KNUXT5JAA  ZKU755
TZRU36TAA  ZRU361
AAA8067  AAA9067
AAH8437  AAM8437
AAN3G

In [24]:
val_training_set = np.array(val_data,dtype=np.float32)
val_training_labels = np.array(val_labels)
val_ragged = tf.ragged.constant(val_training_labels).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices((val_training_set,val_ragged)).shuffle(640).batch(64)

C:\Users\carlos\AppData\Local\Temp/ipykernel_21744/3660101917.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_training_labels = np.array(val_labels)


In [1]:
import cv2
from gen_plates_keras import ImageGenerator, RealImageGenerator

gen = RealImageGenerator()

In [3]:
images,labels = gen.generate_images(100,training=False)
for im,label in zip(images,labels):
    cv2.imshow("window",im)
    print(label)
    cv2.waitKey(0)
cv2.destroyAllWindows()

NGM1306
XTH254
NFI1869
DAA7902
NCW7897
ABD8382
DAD2961
NAH5170
PHO232
NBR9005
0401
DAL3958
B4U769
DAZ9022
DAL7407
NCG1221
NCB4238
ZPD628
DBA3050
DRL923
TNV174
NDM9034
NCV1992
NAQ6323
NEH1815
04011290399
NGB1199
NCC9188
UBG628
05240220800
NEH6898
ABP3663
29OCIQ
AAK6509
NDJ3111
TSL689
NK1258
NAIT3873
NCO1652
XLX969
ABE6647
UGN669
NBG5119
G2R209
AAX2663
TQP549
JCB544
VFN171
NBP8089
WCQ791
AAJ5201
NBW250B
VBT961
NAT6342
NCZ5306
NBG1384
GAO1457
NBZ8792
NBJ355
NCG1092
NAZ2072
AAV9943
CAM5446
NCC1100
NAU6565
WQT879
NH
DCQ9890
NEH7107
NBK6857
AAC2183
AHA2225
NDL1031
NAV0698
B6V113
NCI664
NCI664
NFX7563
DXT488
PQX901
FRK248
NDG3903
1303
AAK2200
UHE795
DXW427
TGG996
ARA6828
DAJ3537
ZSC331
NCG2307
NBP4517
DAA3281
AGA7763
IAC7876
1303
NAA6953
NBH1332
XMV847
NVQ802
